In [1]:
import numpy as np
import tensorflow as tf
import sys

import numpy as np
import os

import matplotlib.pyplot as plt
%matplotlib inline

2022-01-17 12:22:44.284581: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pbs/throng/lsst/users/bbiswas/miniconda3/envs/fvae_miniconda/lib/:
2022-01-17 12:22:44.284632: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
sys.path.insert(0,'../')
from scripts.FlowVAEnet import FlowVAEnet
from scripts.utils import listdir_fullpath
from scripts.utils import norm, denorm

## Load images

In [3]:
bands = [4,5,6,7,8,9]

######## List of data samples
images_dir = '/sps/lsst/users/barcelin/data/isolated_galaxies/' + '27.5/centered/'
list_of_samples = [x for x in listdir_fullpath(os.path.join(images_dir,'training')) if x.endswith('.npy')]
list_of_samples_val = [x for x in listdir_fullpath(os.path.join(images_dir,'validation')) if x.endswith('.npy')]

# Take 1000 images from the first file.
images = np.load(list_of_samples_val[0],  mmap_mode = 'c')[:1000]

normalization_weights_path = '/sps/lsst/users/barcelin/data/blended_galaxies/' + '27.5/centered/test' 
print("Shape of loaded image" + str(np.shape(images)))

Shape of loaded image(1000, 2, 10, 64, 64)


## Convernt image to linear normalization
Note that the images were non-linearly normalized before being saved. So do denormalization first.

In [4]:
# noisy or blended galaxies
images_noisy_normed = convert_to_linear_norm(images[:, 1, 4:])
images_noisy_normed = np.transpose(images_noisy_normed, axes = (0,2,3,1))

# isolated central galaxies
images_isolated_normed = convert_to_linear_norm(images[:, 0, 4:])
images_isolated_normed = np.transpose(images_isolated_normed, axes = (0,2,3,1))

NameError: name 'convert_to_linear_norm' is not defined

In [ ]:
np.shape(images_isolated_normed)

## Load trained models

In [ ]:
flow_net = FlowVAEnet(latent_dim=32, linear_norm=True)
flow_net.load_vae_weights(weights_path='/sps/lsst/users/bbiswas/weights/LSST/FlowDeblender/separated_architecture/vae/')
flow_net.load_flow_weights(weights_path='/sps/lsst/users/bbiswas/weights/LSST/FlowDeblender/separated_architecture/fvae/')
print(flow_net.vae_model.summary())

## Test if VAE works

### Looking at image reconstruction

In [ ]:
model_vae_output_isolated = flow_net.vae_model(images_isolated_normed)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12,4))

image_num = 1

im = axs[0].imshow(images_isolated_normed[image_num][:, :, 2])
fig.colorbar(im, ax=axs[0], shrink=0.8)
axs[0].set_title("original image")

im = axs[1].imshow(model_vae_output_isolated[0].mean().numpy()[image_num][ :, :, 2])
fig.colorbar(im, ax=axs[1], shrink=0.8)
axs[1].set_title("output image")


difference = images_isolated_normed[image_num][:, :, 2] - model_vae_output_isolated[0].mean().numpy()[image_num][ :, :, 2]
im = axs[2].imshow(difference)
fig.colorbar(im, ax=axs[2],shrink=0.8)
axs[2].set_title("difference")

plt.tight_layout()

In [ ]:
plt.imshow(images_noisy_normed[0][ :, :, 2])
plt.title("original r band image");

## Test if flow network works

The likelihood should be higher for isolated galaxies:

In [ ]:
isolated_likelihood = flow_net.flow_model(images_isolated_normed)
noisy_likelihood = flow_net.flow_model(images_noisy_normed)

likelihood_difference=isolated_likelihood-noisy_likelihood

In [ ]:
plt.hist(likelihood_difference.numpy(), bins=30);